# Step 0: Install dependencies

In [ ]:
# !pip install azure-identity==1.15.0
# !pip install azureml-fsspec==1.3.1
# !pip install azureml-sdk==1.55.0
# !pip install azure-ai-ml==1.15.0
# !pip install mltable==1.6.1

# Step 1: Set up your workspace

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "suscription-id-413e-b571-e52d252eb3c6"
    resource_group = "dmc-group"
    workspace = "jose-dmc"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
except Exception as ex:
    print(ex)

# Step 2: Data source and format

In [2]:
import mltable

paths = [
    {'file': './train_data/iris_dataset.csv'}
]

train_table = mltable.from_delimited_files(paths)
train_table.save('./train_data')

paths:
- file: file://d:\DMC\AutoML_Azure\train_data\iris_dataset.csv
transformations:
- read_delimited:
    delimiter: ','
    empty_as_string: false
    encoding: utf8
    header: all_files_same_headers
    include_path_column: false
    infer_column_types: true
    partition_size: 20971520
    path_column: Path
    support_multi_line: false
type: mltable

# Step 3: Create a Compute Instance (If not exist)

In [3]:
# #Sign into Azure with Azure CLI
# !az login

In [5]:
ci_basic_name = "jose-basic-ci"
from azure.ai.ml.entities import ComputeInstance, AmlCompute
ci_basic = ComputeInstance(name=ci_basic_name, size="Standard_DS2_v2")
ml_client.begin_create_or_update(ci_basic).result()


ComputeInstance({'state': 'Running', 'last_operation': {'operation_name': 'Create', 'operation_time': '2024-03-31T13:49:39.065Z', 'operation_status': 'Succeeded', 'operation_trigger': 'User'}, 'os_image_metadata': <azure.ai.ml.entities._compute._image_metadata.ImageMetadata object at 0x0000023493BC8FD0>, 'services': [{'display_name': 'Jupyter', 'endpoint_uri': 'https://jose-basic-ci.eastus.instances.azureml.ms/tree/'}, {'display_name': 'Jupyter Lab', 'endpoint_uri': 'https://jose-basic-ci.eastus.instances.azureml.ms/lab'}], 'type': 'computeinstance', 'created_on': '2024-03-31T13:49:31.881678+0000', 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'jose-basic-ci', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/37520400-632e-413e-b571-e52d252eb3c6/resourceGroups/dmc-group/providers/Microsoft.MachineLearningServices/workspaces/jose-dmc/computes/jose-basic-ci', 'Resource__source_path': '', 'base_path': 'd:\\DMC\\Aut

# Step 4: Configure your experiment settings

In [6]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl, Input

# make an Input object for the training data
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./train_data"
)

# configure the classification job
classification_job = automl.classification(
    compute             = ci_basic_name,
    experiment_name     = "jose_experiment_python_3",
    training_data       = my_training_data_input,
    target_column_name  = "target",
    primary_metric      = "accuracy",
    n_cross_validations = 5,
    enable_model_explainability = True,
    tags                = {"dmc": "demo"}
)

# # Limits are all optional
# classification_job.set_limits(
#     timeout_minutes=600, 
#     trial_timeout_minutes=20, 
#     max_trials=2,
#     enable_early_termination=True,
# )

# # Training properties are optional
# classification_job.set_training(
#     blocked_training_algorithms=["logistic_regression"], 
#     enable_onnx_compatible_models=True
# )

# Step 5: Run experiment

In [7]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(classification_job)

# Get the URL to monitor the job in Azure Machine Learning studio
print(f"Monitor your job at {returned_job.studio_url}")

Monitor your job at https://ml.azure.com/runs/quirky_star_nktb75xkkp?wsid=/subscriptions/37520400-632e-413e-b571-e52d252eb3c6/resourcegroups/dmc-group/workspaces/jose-dmc&tid=1640d5fb-1c9a-4aa4-8d9e-7850838a248a
